### Since Power BIs Mapping Tool doesn't like City and State names, let's try Longitude and Latitude

In [9]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from time import sleep

In [10]:
# Sample
geolocator = Nominatim(user_agent="MyApp")
location = geolocator.geocode("Athens, Georgia")

In [11]:
print("The latitude of the location is: ", location.latitude)
print("The longitude of the location is: ", location.longitude)

The latitude of the location is:  33.9597677
The longitude of the location is:  -83.376398


In [12]:
location

Location(Athens-Clarke County Unified Government, Athens-Clarke County, Georgia, United States, (33.9597677, -83.376398, 0.0))

In [13]:
# This imports the brewery data we will be working with
breweries = pd.read_csv("breweries.csv")
breweries['state'] = breweries['state'].str.strip()

In [14]:
breweries

,Unnamed: 0,name,city,state
0,0,NorthGate Brewing,Minneapolis,MN
1,1,Against the Grain Brewery,Louisville,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA
3,3,Mike Hess Brewing Company,San Diego,CA
4,4,Fort Point Beer Company,San Francisco,CA
...,...,...,...,...
553,553,Covington Brewhouse,Covington,LA
554,554,Dave's Brewfarm,Wilson,WI
555,555,Ukiah Brewing Company,Ukiah,CA
556,556,Butternuts Beer and Ale,Garrattsville,NY


In [15]:
# This will replace the state abbrev with the state name, which should fix some errors
state_code = pd.read_csv('state_code.csv')
breweries = breweries.merge(state_code, left_on='state', right_on='code', how='left')
breweries.head()

,Unnamed: 0,name,city,state_x,state_y,abbrev,code
0,0,NorthGate Brewing,Minneapolis,MN,Minnesota,Minn.,MN
1,1,Against the Grain Brewery,Louisville,KY,Kentucky,Ky.,KY
2,2,Jack's Abby Craft Lagers,Framingham,MA,Massachusetts,Mass.,MA
3,3,Mike Hess Brewing Company,San Diego,CA,California,Calif.,CA
4,4,Fort Point Beer Company,San Francisco,CA,California,Calif.,CA


In [16]:
# This is the cleaner version of the data that we want to work with
breweries = breweries[['name', 'city', 'state_x', 'state_y']].rename(columns={'state_x':'state_code','state_y':'state_name'})
breweries.head()

,name,city,state_code,state_name
0,NorthGate Brewing,Minneapolis,MN,Minnesota
1,Against the Grain Brewery,Louisville,KY,Kentucky
2,Jack's Abby Craft Lagers,Framingham,MA,Massachusetts
3,Mike Hess Brewing Company,San Diego,CA,California
4,Fort Point Beer Company,San Francisco,CA,California


#### This code searches for the coordinates of cities by name through the geopy library, using an if statement to prevent misspellings from breaking the code

In [17]:
# This list will be used to search for the city coordinates by name
city_list = []
for n in range(len(breweries)):
    city = breweries.iloc[n]['city'] + ', ' + breweries.iloc[n]['state_name'] + ', United States'
    city_list.append(city)

In [18]:
# This code seaches for the longitude and latitude, and proves a None for any errors, which will we fix manually
city_long_lat_list = []
for city in city_list:
    city_location = geolocator.geocode(city, timeout=10000)
    if city_location == None:
        city_long_lat_list.append(None)
    else:
        city_long_lat = f"{city_location.longitude} {city_location.latitude}"
        city_long_lat_list.append(city_long_lat)

In [19]:
len(city_list)

558

In [20]:
len(city_long_lat_list)

558

#### This is where I manually fix the misspelled city names and replace the nones in the coordinates

In [21]:
city_long_lat_list.index(None)

293

In [22]:
city_list[293]

'Northamtpon, Massachusetts, United States'

In [23]:
city_list[293] = 'Northampton, Massachusetts, United States'

In [24]:
city_location = geolocator.geocode(city_list[293], timeout=10000)
city_long_lat = f"{city_location.latitude}, {city_location.longitude}"
city_long_lat

'42.3178989, -72.6311006'

In [25]:
city_long_lat_list[293] = '42.3178989 -72.6311006'

In [26]:
city_long_lat_list.index(None)

314

In [27]:
city_list[314]

'Bucryus, Missouri, United States'

In [28]:
city_list[314] = 'Bucyrus, Missouri, United States'

In [29]:
city_location = geolocator.geocode(city_list[314], timeout=10000)
city_long_lat = f"{city_location.latitude}, {city_location.longitude}"
city_long_lat

'37.3442143, -92.0168233'

In [30]:
city_long_lat_list[314] = '37.3442143 -92.0168233'

In [31]:
city_long_lat_list.index(None)

ValueError: None is not in list

### Now we just match up the coordinates with the dataframe and send it back to CSV form for use in Power BI

In [32]:
breweries['city_long_lat'] = city_long_lat_list

In [34]:
breweries[['city_longitude', 'city_latitude']] = breweries['city_long_lat'].str.split(expand=True)

In [35]:
breweries = breweries.drop(columns='city_long_lat')

In [36]:
# And now we've got the Longitudes and Latitudes
breweries.head()

,name,city,state_code,state_name,city_longitude,city_latitude
0,NorthGate Brewing,Minneapolis,MN,Minnesota,-93.2654692,44.9772995
1,Against the Grain Brewery,Louisville,KY,Kentucky,-85.759407,38.2542376
2,Jack's Abby Craft Lagers,Framingham,MA,Massachusetts,-71.4165905,42.2773177
3,Mike Hess Brewing Company,San Diego,CA,California,-117.1627728,32.7174202
4,Fort Point Beer Company,San Francisco,CA,California,-122.419906,37.7790262


In [37]:
breweries.to_csv("cleaned_breweries.csv")